In [ ]:
!pip uninstall -y eonacs && pip install git+https://github.com/njnmco/smc #--log /dev/stderr

In [2]:
import pandas as pd
import numpy as np

In [3]:
import eonacs.common.util as util

util.colab_map_drive()


Mounted at /content/drive


In [4]:
model_path = "/content/drive/MyDrive/smc/models/dbert_tasks_pac_refactor"

In [5]:
import eonacs.common.dbert as dbert

tokenizer, model = dbert.tebert(model_path)

In [6]:
import json

course = json.load(open('/content/drive/MyDrive/smc/data/curricunet/3064?reportId=5.json') )
course

{'course': 'COMPUTER SCIENCE 34A, Game Programming 1',
 'course_objectives': ['Demonstrate an understanding and applications of the asset workflow model',
  'Create a game play',
  'Use 2D and graphics in game development',
  'Use 3D physics in game development',
  'Apply event driven programming to develop games',
  'Apply audio, music and animation in navigation',
  'Adapt games for different platforms such as Mobile devices'],
 'file': 'data/curricunet/3064?reportId=5',
 'student_learning_outcomes': ['Using the principles of Programming, graphics in 2D and 3D and animation, students will develop interactive video games.',
  'Using the tools of a game development tool, students will debug code for logical, syntax and runtime errors.']}

In [36]:
course_text = pd.Series(course['course_objectives'] + course['student_learning_outcomes'])
course_coords = dbert.get_embeddings(course_text, tokenizer, model)


Using cuda:0
0...


In [8]:
con = util.onet_db()

In [9]:
tasks = pd.read_sql("select onetsoc_code, task from task_statements where task_type = 'Core'",   con)


In [10]:
import eonacs.common.score as score

In [11]:
task_embeddings =  dbert.get_embeddings(tasks["task"], tokenizer, model)

Using cuda:0
0...
1024...
2048...
3072...
4096...
5120...
6144...
7168...
8192...
9216...
10240...
11264...
12288...


In [12]:
task_d_sample = score.distance_sample(task_embeddings)

In [27]:
pd.read_sql("select * from occupation_data where onetsoc_code like '15-125%'", con)

,onetsoc_code,title,description
0,15-1251.00,Computer Programmers,"Create, modify, and test the code and scripts ..."
1,15-1252.00,Software Developers,"Research, design, and develop computer and net..."
2,15-1253.00,Software Quality Assurance Analysts and Testers,Develop and execute software tests to identify...
3,15-1254.00,Web Developers,"Develop and implement websites, web applicatio..."
4,15-1255.00,Web and Digital Interface Designers,Design digital user interfaces or websites. De...
5,15-1255.01,Video Game Designers,Design core features of video games. Specify i...


In [47]:
se_scores = score.qdist(task_d_sample, course_coords)[tasks["onetsoc_code"] == '15-1251.00', :]

In [48]:
se_scores

array([[ 89.,  81.,  97.,  97.,  94.,  94., 100.,  62.,  94.],
       [ 48.,  96.,  99.,  98.,  81., 100.,  98.,  70.,  96.],
       [ 75.,  19.,  29.,  29.,  30.,  24.,  51.,  23.,  24.],
       [ 75.,  31.,  64.,  60.,  49.,  52.,  87.,  14.,  38.],
       [ 71.,  86.,  84.,  88.,  96.,  81.,  77.,  99.,  96.],
       [ 66.,  22.,  43.,  39.,  31.,  35.,  68.,  16.,  31.],
       [ 66.,  35.,  67.,  62.,  44.,  56.,  91.,  16.,  47.],
       [ 81.,  90.,  99.,  98.,  90.,  97.,  99.,  50.,  92.],
       [ 70.,  83.,  81.,  83.,  96.,  79.,  74., 100.,  94.],
       [ 86.,  23.,  44.,  38.,  43.,  30.,  72.,  25.,  30.],
       [ 91.,  79.,  99.,  99.,  97.,  91., 100.,  58.,  85.],
       [ 77.,  19.,  48.,  44.,  36.,  31.,  67.,  18.,  29.],
       [ 78.,  34.,  43.,  40.,  61.,  35.,  56.,  47.,  44.]])

In [49]:
# Occupation tasks covered by course
pd.DataFrame({"Occupational task":tasks.loc[tasks["onetsoc_code"] == '15-1251.00', "task"], "Score":se_scores.max(axis=1)})

,Occupational task,Score
1716,"Write, analyze, review, and rewrite programs, ...",100.0
1717,Correct errors by making appropriate changes a...,100.0
1718,"Perform or direct revision, repair, or expansi...",75.0
1719,"Write, update, and maintain computer programs ...",87.0
1720,"Consult with managerial, engineering, and tech...",99.0
1721,Conduct trial runs of programs and software ap...,68.0
1722,Prepare detailed workflow charts and diagrams ...,91.0
1723,Compile and write documentation of program dev...,99.0
1724,Consult with and assist computer operators or ...,100.0
1725,Perform systems analysis and programming tasks...,86.0


Now, compare with Video Game Programmer

In [50]:
vgp_scores = score.qdist(task_d_sample, course_coords)[tasks["onetsoc_code"] == '15-1255.01', :]

In [51]:
pd.DataFrame({"Occupational task":tasks.loc[tasks["onetsoc_code"] == '15-1255.01', "task"], "Score":vgp_scores.max(axis=1)})

,Occupational task,Score
1778,Balance and adjust gameplay experiences to ens...,99.0
1779,Provide feedback to designers and other collea...,99.0
1780,"Create core game features, including storyline...",99.0
1781,"Devise missions, challenges, or puzzles to be ...",100.0
1782,Guide design discussions between development t...,99.0
1783,Develop and maintain design level documentatio...,100.0
1784,"Create and manage documentation, production sc...",100.0
1785,Present new game design concepts to management...,99.0
1786,Conduct regular design reviews throughout the ...,100.0
1787,"Solicit, obtain, and integrate feedback from d...",100.0


In [52]:
# Syllabus items' relevancy to occ tasks 
pd.DataFrame({"Syllabus item":course_text, "Computer Programmer Score":se_scores.max(axis=0), "Video Game Programmer Score": vgp_scores.max(axis=0) })

,Syllabus item,Computer Programmer Score,Video Game Programmer Score
0,Demonstrate an understanding and applications ...,91.0,100.0
1,Create a game play,96.0,100.0
2,Use 2D and graphics in game development,99.0,100.0
3,Use 3D physics in game development,99.0,100.0
4,Apply event driven programming to develop games,97.0,100.0
5,"Apply audio, music and animation in navigation",100.0,99.0
6,Adapt games for different platforms such as Mo...,100.0,100.0
7,"Using the principles of Programming, graphics ...",100.0,100.0
8,"Using the tools of a game development tool, st...",96.0,100.0
